## Package

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import MNIST, FashionMNIST

from torch.utils.data import DataLoader

from architectures_28x28.CKAN_BN import CKAN_BN
from architectures_28x28.SimpleModels import *
from architectures_28x28.ConvNet import ConvNet
from architectures_28x28.KANConvs_MLP import KANC_MLP
from architectures_28x28.KKAN import KKAN_Convolutional_Network
from architectures_28x28.conv_and_kan import NormalConvsKAN


## Dataset

In [2]:
# Transformaciones
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

mnist_test = MNIST(root='./data', train=False, download=True, transform=transform)
fashion_mnist_test = FashionMNIST(root='./data', train=False, transform=transform, download=True)

fashion_mnist_test_loader = DataLoader(fashion_mnist_test, batch_size=64, shuffle=False)
mnist_test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

## Set Device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Load model

In [23]:
def load_model(model_class, model_path):
    model = model_class()
    model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))
    model.to(device)
    model.eval()
    return model


In [20]:
def calculate_metrics(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)  # Ensure data and target are on the correct device
            output = model(data)
            preds = output.argmax(dim=1, keepdim=True).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(target.cpu().numpy())
    
    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels).flatten()
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    return accuracy, precision, recall, f1



def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [39]:
models = {
    'SimpleCNN_2': 'models/SimpleCNN_2/best_model.pth',
    'SimpleCNN': 'models/SimpleCNN/best_model.pth',
    'SimpleLinear': 'models/SimpleLinear/best_model.pth',
    'ConvNet': 'models/ConvNet/best_model.pth',
    # 'KKAN_Convolutional_Network': 'models/KKAN_Convolutional/best_model.pth',
    # 'KANC_MLP': 'models/KANC_MLP/best_model.pth',
    'NormalConvsKAN': 'models/NormalConvsKAN/best_model.pth',
}


In [40]:
mnist_results = []
for model_name, model_path in models.items():
    model_class = globals()[model_name]
    model = load_model(model_class, model_path)
    
    mnist_metrics = calculate_metrics(model, mnist_test_loader, device)
    num_params = count_parameters(model)
    
    mnist_results.append({
        'Model': model_name,
        'Accuracy': mnist_metrics[0],
        'Precision': mnist_metrics[1],
        'Recall': mnist_metrics[2],
        'F1 Score': mnist_metrics[3],
        'Parameters': num_params
    })
    
# Create DataFrames
df_mnist_results = pd.DataFrame(mnist_results)

# Style the DataFrames
df_mnist_styled = df_mnist_results.style.format({
    'Accuracy': '{:.4f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}',
    'F1 Score': '{:.4f}',
    'Parameters': '{:,}'
}).set_caption("Model Evaluation Metrics on MNIST Dataset")


# Display the styled DataFrames
df_mnist_styled

,Model,Accuracy,Precision,Recall,F1 Score,Parameters
0,SimpleCNN_2,0.9907,0.9907,0.9907,0.9907,"157,030"
1,SimpleCNN,0.9728,0.9728,0.9728,0.9728,"2,740"
2,SimpleLinear,0.9203,0.9206,0.9203,0.9200,"7,850"
3,ConvNet,0.9958,0.9958,0.9958,0.9958,"887,530"
4,NormalConvsKAN,0.9803,0.9803,0.9803,0.9803,"37,030"


In [41]:
fashion_mnist_results = []

for model_name, model_path in models.items():
    model_class = globals()[model_name]
    model = load_model(model_class, model_path)
    
    fashion_mnist_metrics = calculate_metrics(model, fashion_mnist_test_loader, device)
    num_params = count_parameters(model)
    
   
    fashion_mnist_results.append({
        'Model': model_name,
        'Accuracy': fashion_mnist_metrics[0],
        'Precision': fashion_mnist_metrics[1],
        'Recall': fashion_mnist_metrics[2],
        'F1 Score': fashion_mnist_metrics[3],
        'Parameters': num_params
    })

# Create DataFrames
df_fashion_mnist_results = pd.DataFrame(fashion_mnist_results)

# Style the DataFrames

df_fashion_mnist_styled = df_fashion_mnist_results.style.format({
    'Accuracy': '{:.4f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}',
    'F1 Score': '{:.4f}',
    'Parameters': '{:,}'
}).set_caption("Model Evaluation Metrics on Fashion-MNIST Dataset")

# Display the styled DataFrames
df_fashion_mnist_styled

,Model,Accuracy,Precision,Recall,F1 Score,Parameters
0,SimpleCNN_2,0.1712,0.1905,0.1712,0.1618,"157,030"
1,SimpleCNN,0.1838,0.1351,0.1838,0.1260,"2,740"
2,SimpleLinear,0.0901,0.0803,0.0901,0.0636,"7,850"
3,ConvNet,0.1296,0.1808,0.1296,0.1227,"887,530"
4,NormalConvsKAN,0.0754,0.1418,0.0754,0.0696,"37,030"
